In [2]:
from townsnet import Region

regions_dict = {
  1: 'Ленинградская область',
  3138: 'Санкт-Петербург',
  3268: 'Москва',
  3427: 'Волгоградская область',
  3902: 'Тульская область',
  4013: 'Омская область',
  4437: 'Краснодарский край',
  4882: 'Тюменская область',
  5188: 'Московская область',
}

regions = {region_id:Region.from_pickle(f'data/regions/{region_id}/{region_id}.pickle') for region_id in regions_dict.keys()}

In [3]:
for region_id, region in regions.items():
  print(region_id, len(region.get_services_gdf()))

1 26018
3138 55892
3268 109623
3427 14784
3902 11545
4013 11951
4437 51867
4882 15139
5188 97400


In [18]:
from townsnet import Provision
from tqdm import tqdm
from multiprocessing import Pool

for region_id, region in tqdm(regions.items()):
  provision = Provision(region=region)
  
  def calculate_provision(st):
    data_path = f'data/regions/{region_id}/provisions/'
    d_gdf, s_gdf, t_gdf, l_gdf = provision.calculate(st)
    t_gdf['settlement_name'] = t_gdf['settlement_name'].apply(lambda v : None if v==0 else v)
    t_gdf['district_name'] = t_gdf['district_name'].apply(lambda v : None if v==0 else v)
    d_gdf.to_parquet(data_path + f'{st.name}_districts.parquet')
    s_gdf.to_parquet(data_path + f'{st.name}_settlements.parquet')
    t_gdf.to_parquet(data_path + f'{st.name}_towns.parquet')
    l_gdf.to_parquet(data_path + f'{st.name}_links.parquet')
  
  with Pool() as pool:
    pool.map(calculate_provision, region.service_types)

100%|██████████| 9/9 [04:58<00:00, 33.13s/it]


In [75]:
import geopandas as gpd
import pandas as pd

source = 'https://www.openstreetmap.org/'
period = '2023'

for region_id, region_name in regions_dict.items():
  data_path = f'data/regions/{region_id}/'
  region = regions[region_id]
  st_df = region.get_service_types_df()
  values = []
  for i,infrastructure in enumerate(st_df.infrastructure.unique()):
    st_sub_df = st_df[st_df.infrastructure == infrastructure]
    for j, st_name in enumerate(st_sub_df.name):
      code = f'5.{i+1}.{j+1}'
      d_gdf = gpd.read_parquet(data_path + f'provisions/{st_name}_districts.parquet')
      s_gdf = gpd.read_parquet(data_path + f'provisions/{st_name}_settlements.parquet')
      t_gdf = gpd.read_parquet(data_path + f'provisions/{st_name}_towns.parquet')
      l_gdf = gpd.read_parquet(data_path + f'provisions/{st_name}_links.parquet')
      total = Provision.total_provision(d_gdf)
      values.append({
        '№ п/п': code,
        'Значение': round(total,2) if total is not None else None,
        'Территория': region_name,
        'Источник': source,
        'Период': period,
      })
      for district_id in d_gdf.index:
        district_name = d_gdf.loc[district_id, 'name']
        total = d_gdf.loc[district_id, 'provision']
        values.append({
          '№ п/п': code,
          'Значение': round(total,2) if total is not None else None,
          'Территория': district_name,
          'Источник': source,
          'Период': period,
        })
      for settlement_id in s_gdf.index:
        settlement_name = s_gdf.loc[settlement_id, 'name']
        total = s_gdf.loc[settlement_id, 'provision']
        values.append({
          '№ п/п': code,
          'Значение': round(total,2) if total is not None else None,
          'Территория': settlement_name,
          'Источник': source,
          'Период': period,
        })
  values_df = pd.DataFrame(values)
  values_df.to_excel(data_path+f'{region_id} {region_name}.xlsx', index=False)

In [73]:
values_df.to_excel('test.xlsx', index=False)

In [23]:
region = regions[1]